<a href="https://colab.research.google.com/github/Gresey/Machine-Learning-Lab/blob/main/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.11538099,  0.5373011 , -0.01277626,  0.6025788 , -0.7150549 ,
        -0.30943987,  0.61999977, -0.4848336 ,  0.51138717,  0.12857062]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.07475625, 0.14358324, 0.08283391, 0.15326871, 0.04104048,
        0.06156995, 0.15596218, 0.05166498, 0.13991022, 0.0954101 ]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

2.7875814

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [ ]:
model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2999 - accuracy: 0.9136
Epoch 2/3
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1448 - accuracy: 0.9567
Epoch 3/3
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1093 - accuracy: 0.9660


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0851 - accuracy: 0.9731 - 807ms/epoch - 3ms/step


[0.0851358100771904, 0.9731000065803528]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.63332481e-07, 4.01076932e-08, 2.01552262e-04, 3.18256236e-04,
        1.41009382e-09, 3.57863286e-08, 7.81744623e-13, 9.99457061e-01,
        5.58045087e-07, 2.22060135e-05],
       [1.69164920e-07, 5.48815937e-04, 9.99434769e-01, 1.35249102e-05,
        1.16964121e-12, 1.19444530e-06, 3.38255148e-07, 2.22509511e-12,
        1.24646567e-06, 6.23319484e-14],
       [2.51667529e-06, 9.98711944e-01, 1.92456457e-04, 3.54358344e-05,
        1.87591904e-05, 4.45842961e-05, 6.83767721e-05, 7.49777188e-04,
        1.70969390e-04, 5.25126779e-06],
       [9.99105990e-01, 2.15296829e-08, 8.47700503e-05, 1.55986272e-05,
        3.66697975e-07, 2.22034869e-05, 6.32058363e-04, 1.53391975e-05,
        2.30435467e-06, 1.21267556e-04],
       [3.69657437e-06, 1.40218731e-07, 2.33858082e-05, 8.27617555e-07,
        9.96563494e-01, 5.57371914e-06, 8.98345679e-05, 4.00186836e-05,
        9.88675492e-06, 3.26320692e-03]], dtype=float32)>